In [1]:
import ee

In [2]:
from geetools import ui

In [3]:
p = ee.Geometry.Point([-72, -42])

In [4]:
import threading
import multiprocessing

In [5]:
from IPython.display import display
import ipywidgets as widgets
infowin = widgets.Output()

def get_info(eeobj):
    info = eeobj.getInfo()
    # html = widgets.HTML(str(info))
    wid = ui.ipytools.create_accordion(info)
    with infowin:
        display(wid)
        
thread = threading.Thread(target=get_info, args=(p,))
display(infowin)
thread.start()

Output()

In [6]:
def eprint(*args, indent=2, notebook=True, do_async=False):
    """ Print an EE Object. Same as `print(object.getInfo())`

    :param eeobject: object to print
    :type eeobject: ee.ComputedObject
    :param notebook: if True, prints the object as an Accordion Widget for
        the Jupyter Notebook
    :type notebook: bool
    :param indent: indentation of the print output
    :type indent: int
    :param do_async: call getInfo() asynchronously
    :type do_async: bool
    """

    import pprint
    pp = pprint.PrettyPrinter(indent=indent)
    
    from IPython.display import display
    from ipywidgets import Output, HTML
    infowin = Output()
    
    info_return = []

    def get_info(eeobject, from_ee, infowindow):
        """ Get Info and append it to infowindow """
        if from_ee:
            info = eeobject.getInfo()
        else:
            info = eeobject
        
        if notebook:
            widget = HTML(str(info)+'<br/>')        
            with infowindow:
                display(widget)
            info_return.append(widget)
        else:
            print(info)
            info_return.append(info)

    for eeobject in args:
        module = getattr(eeobject, '__module__', None)
        parent = module.split('.')[0] if module else None
        if parent == ee.__name__:
            if do_async:
                thread = threading.Thread(target=get_info, args=(eeobject, True, infowin))
                thread.start()
            else:
                get_info(eeobject, True, infowin)
        else:
            get_info(eeobject, False, infowin)
    
    display(infowin)
    return info_return

In [7]:
data = eprint(p, 'test', p, do_async=True, notebook=True)

Output()

In [8]:
import threading
import queue

# https://stackoverflow.com/questions/35160417/threading-queue-working-example

def worker(*args, notebook):
    class Worker(threading.Thread):
        def __init__(self, q, *args, **kwargs):
            self.q = q
            super().__init__(*args, **kwargs)

        def run(self):
            while True:
                try:
                    work = self.q.get(timeout=3)  # 3s timeout
                except queue.Empty:
                    return
                # do whatever work you have to do on work
                print(work.getInfo())
                self.q.task_done()

    b = [ee.Image.constant(0).add(10000).subtract(10000), p, ee.Number(1)]


    q = queue.Queue()

    for eeobj in args:
        q.put(eeobj)

    for _ in range(len(args)):
        Worker(q).start()
        # q.join()  # blocks until the queue is empty.

In [9]:
class Worker(threading.Thread):
    def __init__(self, q, *args, **kwargs):
        self.q = q
        super().__init__(*args, **kwargs)

    def run(self):
        while True:
            try:
                work = self.q.get(timeout=3)  # 3s timeout
            except queue.Empty:
                return
            # do whatever work you have to do on work
            print(work.getInfo())
            self.q.task_done()

b = [ee.Image.constant(0).add(10000).subtract(10000), p, ee.Number(1)]

q = queue.Queue()

for eeobj in b:
    q.put(eeobj)

for _ in range(len(b)):
    Worker(q).start()
    # q.join()  # blocks until the queue is empty.

{'geodesic': True, 'coordinates': [-72.0, -42.0], 'type': 'Point'}
{'type': 'Image', 'bands': [{'id': 'constant', 'crs_transform': [1.0, 0.0, 0.0, 0.0, 1.0, 0.0], 'data_type': {'max': 0, 'type': 'PixelType', 'precision': 'int', 'min': 0}, 'crs': 'EPSG:4326'}]}
1
